In [32]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import csv
import numpy as np
import random
from sklearn.utils import shuffle
import jieba

In [33]:
data = pd.read_csv('./data/train_test_data.csv',encoding="utf8",error_bad_lines=False, sep=None)
data=data[['content','target']]
data = data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data['target'] =data['target'].astype("int")
data = shuffle(data)#乱序

/Users/exudingtao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [34]:
tea = data[data['target']==0]
tea = tea['content'].values.tolist()

In [35]:
leaner = data[data['target']==1]
leaner = leaner['content'].values.tolist()

In [36]:
len(tea)

750

In [37]:
len(leaner)

814

In [1]:
#去停用词
stop_word_path = './data/stopword.txt'
stop_word_list = [sw.replace('\n','') for sw in open(stop_word_path).readlines()]

def preprocess_text(content_lines,sentences,category):
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            #segs = list(filter(lambda x: len(x)>1,segs))
            segs = list(filter(lambda x: x not in stop_word_list,segs))
            sentences.append((' '.join(segs),category))#分完词后 用空格串起来 且和label合并为一个元组
        except Exception as e:
            print(line)
            continue

In [39]:
#生成训练数据
sentences = []
preprocess_text(leaner,sentences,'leaner')
preprocess_text(tea,sentences,'tea')

In [40]:
random.shuffle(sentences)

In [41]:
from sklearn.model_selection import train_test_split
x,y = zip(*sentences)
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1234)

In [42]:
#交叉验证
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score

In [43]:
seed = 10
def stratifiedFfold_cv(x,y,clf_class,folds=5,**kwargs):
    folds = KFold(n_splits=folds, shuffle=True, random_state=seed)
    y_pred = y[:]
    for fold, (train_index, test_index) in enumerate(folds.split(x, y)):
        x_train,x_test = x[train_index],x[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(x_train,y_train)
        y_pred[test_index] = clf.predict(x_test)
    return y_pred

In [44]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
class TextClassifierSVM():
    def __init__(self,classifier = SVC(kernel='linear')):#可以换内核
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,4),max_features=20000)
    def features(self,X):
        return self.vectorizer.transform(X)
    def fit(self,X,y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X),y)
    def predict(self,X):
        return self.classifier.predict(self.features([X]))
    def score(self,X,y):
        return self.classifier.score(self.features(X),y)

In [50]:
text_classifier = TextClassifierSVM()
text_classifier.fit(x_train,y_train)
print(text_classifier.predict('我是一个mooc的学习者'))
print(text_classifier.score(x_test,y_test))

['tea']
0.8516624040920716


In [20]:
####用于聚类

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec=TfidfVectorizer(
    analyzer='word', 
    ngram_range=(1,4),
    max_features=20000
    )
vec.fit(x_train)                       #从新训练集fit特征

'''
使用TF-IDF提取特征，结果为0.8755
'''
tf_idf = vec.transform(x_train)

In [22]:
# 获取词袋模型中的所有词语
word = vec.get_feature_names()  

In [23]:
# 将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重  
weight = tf_idf.toarray()  

In [24]:
#pca降维
from sklearn.decomposition import PCA
pca = PCA(n_components = 800) # 初始化PCA  dimension=800 
weight_of_dim = pca.fit_transform(weight) # 返回降维后的数据  

In [25]:
from sklearn.cluster import KMeans
clusterer = KMeans(n_clusters=10, init='k-means++') # 设置聚类模型 k=10
y = clusterer.fit_predict(weight_of_dim) # 把weight矩阵扔进去fit一下,输出label  

In [26]:
y

array([3, 3, 1, ..., 2, 1, 3], dtype=int32)

In [27]:
len(y)

1173

In [28]:
len(x_train)

1173

In [29]:
set(y)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [49]:
####################------predict------###############
data2 = pd.read_csv('./data/0121-0210 new.csv')

/Users/exudingtao/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [77]:
res[0]

'tea'

In [83]:
#去停用词
stop_word_path = './data/stopword.txt'
stop_word_list = [sw.replace('\n','') for sw in open(stop_word_path).readlines()]
def predict_content(content_lines):
    try:
        segs = jieba.lcut(content_lines)
        segs = list(filter(lambda x: x not in stop_word_list,segs))
        word_list = ' '.join(segs)#分完词后 用空格串起来 且和label合并为一个元组
        res = text_classifier.predict(word_list)[0]
    except:
        res = 'error'
    return res

In [84]:
data2['老师or学生'] = data2['微博'].apply(lambda x:predict_content(x))

In [86]:
data2.to_csv('./data/result.csv',index=False)